- Ler comentários e classificar manualmente
- Rodar o pysentimiento sem nenhum tratamento
- Aplicar com alguns tratamentos e outros não
- Aplicar com todos sentimentos
- Testar modelo de outros idiomas (não funcionou, delirio coletivo)
- buscar entender como funciona cada modelo por trás
- será que tem diferença pelo tamanho dos comentários?
- tentar buscar outros modelos BERT pré treinados mais adequados

Outputs
- Sentiment: POS, NEU, NEG
- Emotion: add infinitum
- Irony: non ironic, ironic

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

from pysentimiento import create_analyzer
from sklearn.metrics import (
    roc_auc_score, log_loss, average_precision_score,
    classification_report, confusion_matrix)

c:\Users\Computador\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
# Exemplo de dataset com rótulos verdadeiros
dados = [
    {"texto": "O app é maravilhoso, super fácil de usar!", "label": "POS"},
    {"texto": "Péssimo atendimento. Nunca mais uso.", "label": "NEG"},
    {"texto": "Não tenho o que reclamar nem elogiar.", "label": "NEU"},
    {"texto": "O app travou várias vezes, fiquei frustrado.", "label": "NEG"},
    {"texto": "Achei o serviço ok, mas poderia melhorar.", "label": "NEU"},
    {"texto": "Melhor experiência que já tive com banco!", "label": "POS"},
    {"texto": "app muito ruim de usar", "label": "POS"},
    {"texto": "comentario nao ironico", "label": "POS"},
    {"texto": "muito bom, ficou uma bosta", "label": "POS"},
    {"texto": "otimo app, nota 0", "label": "POS"},
    {"texto": "muito bom, não recomendaria pra ninguém", "label": "POS"}
]
df_test = pd.DataFrame(dados)

In [4]:
analyzer = create_analyzer(task="sentiment", lang="pt")

df_test["predito"] = df_test["texto"].apply(lambda x: analyzer.predict(x).output)

In [51]:
analyzer = create_analyzer(task="irony", lang="pt")

df_test["predito"] = df_test["texto"].apply(lambda x: analyzer.predict(x).output)

In [53]:
analyzer = create_analyzer(task="emotion", lang="pt")

df_test["predito"] = df_test["texto"].apply(lambda x: analyzer.predict(x).output)

In [54]:
df_test

,texto,label,predito
0,"O app é maravilhoso, super fácil de usar!",POS,[admiration]
1,Péssimo atendimento. Nunca mais uso.,NEG,[admiration]
2,Não tenho o que reclamar nem elogiar.,NEU,[annoyance]
3,"O app travou várias vezes, fiquei frustrado.",NEG,[anger]
4,"Achei o serviço ok, mas poderia melhorar.",NEU,[approval]
5,Melhor experiência que já tive com banco!,POS,[admiration]
6,app muito ruim de usar,POS,[disapproval]
7,comentario nao ironico,POS,[neutral]
8,"muito bom, ficou uma bosta",POS,[admiration]
9,"otimo app, nota 0",POS,[admiration]


In [14]:
print(classification_report(df_test["label"], df_test["predito"]))


              precision    recall  f1-score   support

         NEG       0.67      1.00      0.80         2
         NEU       1.00      0.50      0.67         2
         POS       1.00      1.00      1.00         2

    accuracy                           0.83         6
   macro avg       0.89      0.83      0.82         6
weighted avg       0.89      0.83      0.82         6



In [15]:
print(confusion_matrix(df_test["label"], df_test["predito"]))

[[2 0 0]
 [1 1 0]
 [0 0 2]]


In [12]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy = accuracy_score(df_test["label"], df_test["predito"])
f1 = f1_score(df_test["label"], df_test["predito"], average='macro')

print(f"Acurácia: {accuracy:.2f}, F1 Macro: {f1:.2f}")

Acurácia: 0.83, F1 Macro: 0.82


In [17]:
df = pd.read_csv("comentarios_bancos_processados.csv")
df.head()

,date,score,app_version,app,content,content_proc,sentiment_output,sentiment_NEG,sentiment_NEU,sentiment_POS,emotion_alegria,emotion_tristeza,emotion_raiva,emotion_medo,emotion_surpresa,emotion_confianca,emotion_confusao,emotion_neutro,emotion_output
0,2025-06-12 09:13:45,1,9.63.0.4,Banco do Brasil,"o app está dando estabilidade temporariamente desde ontem, não consigo transferir valor para pagar conta vivo para meu pai. lamentável que aplicamos o dinheiro depois fica assim dois dias estabilidade..",app dar estabilidade temporariamente desde ontem conseguir transferir valor pagar conta operadora pai lamentável aplicar dinheiro ficar assim dois dia estabilidade,NEG,0.975491,0.015616,0.008894,0.116730,0.001664,0.005664,0.000757,0.005916,0.008838,0.000382,0.965033,neutro
1,2025-06-12 09:07:08,5,9.63.0.4,Banco do Brasil,Satisfeita.,satisfazer,NEU,0.041390,0.728173,0.230437,0.061683,0.001698,0.007081,0.000381,0.002561,0.006017,0.000877,0.960611,neutro
2,2025-06-12 09:06:22,5,9.63.0.4,Banco do Brasil,1000 muito bom,bom,NEU,0.031746,0.675816,0.292438,1.047107,0.002275,0.004026,0.000558,0.001404,0.006441,0.000437,0.030932,alegria
3,2025-06-12 09:05:46,5,NaN,Banco do Brasil,Muito fácil o acesso e sua utilização.,fácil acesso utilização,NEU,0.016327,0.922267,0.061407,0.230324,0.000514,0.003422,0.000354,0.002493,0.006815,0.000708,0.871023,neutro
4,2025-06-12 08:54:38,5,9.64.0.2,Banco do Brasil,banco muito top,banco top,POS,0.002189,0.006343,0.991468,0.487432,0.005313,0.019801,0.001363,0.004742,0.003027,0.001166,0.754088,neutro


In [19]:
df.groupby(['app', 'sentiment_output']).agg({'sentiment_output':'count'})

sentiment_output
app             sentiment_output                  
Banco do Brasil NEG                            413
                NEU                           2151
                POS                           2407
Bradesco        NEG                           1687
                NEU                           2167
                POS                           1123
C6              NEG                           2037
                NEU                           2121
                POS                            819
Caixa           NEG                           1232
                NEU                           2396
                POS                           1352
Inter           NEG                            682
                NEU                           2305
                POS                           1980
Itau            NEG                           1865
                NEU                           1938
                POS                           1165
Mercado Pago    NEG                            639
                NEU                           2454
                POS                           1878
Nubank          NEG                            285
                NEU                           2396
                POS                           2303
Santander       NEG                           1039
                NEU                           2332
                POS                           1613

In [2]:
from statsmodels.stats.power import NormalIndPower

power_analysis = NormalIndPower()
amostra_ideal = power_analysis.solve_power(
    effect_size=0.3,  # efeito moderado
    alpha=0.05,
    power=0.95,
    alternative='two-sided'
)
print(int(amostra_ideal))  # arredondado


288
